In [1]:
!pip freeze | grep scikit-learn

scikit-learn @ file:///tmp/build/80754af9/scikit-learn_1642617106979/work
scikit-learn-intelex==2021.20220215.212715


In [2]:
import pickle
import pandas as pd

In [3]:
with open('model.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)

In [4]:
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [5]:
year = 2021
month = 2
output_file = f'fhv_tripdata_{year:04d}-{month:02d}_predicted.parquet'

In [6]:
df = read_data(f'https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_{year:04d}-{month:02d}.parquet')

In [7]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)

In [8]:
# Q1. Notebook
y_pred.mean()

16.191691679979066

In [9]:
# Q2. Preparing the output
df_result = pd.DataFrame()
df_result['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
df_result['predicted_duration'] = y_pred

df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [10]:
!ls -al


total 19316
drwxrwxr-x 3 azureuser azureuser     4096 Jun 21 17:41 .
drwxrwxr-x 8 azureuser azureuser     4096 Jun 18 20:43 ..
drwxrwxr-x 2 azureuser azureuser     4096 Jun 18 20:47 .ipynb_checkpoints
-rw-rw-r-- 1 azureuser azureuser       72 Jun 18 20:43 Dockerfile
-rw-rw-r-- 1 azureuser azureuser      170 Jun 21 17:41 Pipfile
-rw-r--r-- 1 azureuser azureuser     8614 Jun 21 17:41 Pipfile.lock
-rw-rw-r-- 1 azureuser azureuser 19711507 Jun 21 17:47 fhv_tripdata_2021-02_predicted.parquet
-rw-rw-r-- 1 azureuser azureuser    17760 Jun 18 20:43 model.bin
-rw-rw-r-- 1 azureuser azureuser     4578 Jun 18 21:33 starter.ipynb
-rw-rw-r-- 1 azureuser azureuser     1373 Jun 18 21:31 starter.py


In [11]:
# jupyter nbconvert --to script starter.ipynb